This Jupyter notebook employs the sympy Python module to calculate the quantum speed limit, also known as the Mandelstam-Tamm bound and the Margolus-Levitin bound. The calculations presented are based on straightforward examples and re-calculations derived from well-established research papers (see 
<font color = green> https://iopscience.iop.org/journal/1751-8121 </font>
and
<font color = blue> https://journals.aps.org/prl/pdf/10.1103/PhysRevLett.103.240501 </font> ).


In [ ]:
import sympy as sp
import numpy as np


from sympy import Matrix as mx

from sympy import I, BlockMatrix,eye

from sympy.physics.quantum import TensorProduct as tp

from sympy.physics.quantum import Dagger,OuterProduct

from sympy import symbols, pi


ω = sp.Symbol('ω', real=True, positive = True)
ω1 = sp.Symbol('ω1', real=True, positive = True)
ω2 = sp.Symbol('ω2', real=True, positive = True)
ω3 = sp.Symbol('ω3', real=True, positive = True)

κ = sp.Symbol('κ', real=True, positive = True)
κ1 = sp.Symbol('κ1', real=True, positive = True)
κ2 = sp.Symbol('κ2', real=True, positive = True)
κ3 = sp.Symbol('κ3', real=True, positive = True)

In [25]:
# Here, every function needed for the calculation of the quantum speed limit will be defined
# those are: expectation values, moments, standard deviations, arccos functions

# Expectation Value
def expec(Ham,state):
    expec_H_state = (Dagger(state)*Ham*state)[0,0]
    return expec_H_state

# Second Moment
def sec_moment(Ham,state):
    sec_moment_H_state = (Dagger(state)*(Ham**(2))*state)[0,0]
    return sec_moment_H_state

# Standard Deviation
def deviation(Ham,state):
    
    expec_H_state = (expec(Ham,state))
    
    sec_moment_H_state = (sec_moment(Ham,state))
    
    dev1 = sp.simplify( sec_moment_H_state - (expec_H_state)**(2))
    dev2 = sp.sqrt(dev1)
    
    if dev2 == 0.0:
        print("DEVIATION IS ZERO")
    
    return dev2

# Fubini Study Metric, Bures Angle
def angle(ψ0,ψ1):
    
    overlap =  float( (ψ0.H * ψ1)[0] )
    
    abs_overlap = abs( overlap )
    
    angle = np.arccos(abs_overlap)
    
    return angle

# Mandelstamm-Tamm Bound
def mandelstamm(Ham,ψ0,ψ1):
    dev = deviation(Ham, ψ0)
    τ1 = (1/dev)*angle(ψ0,ψ1)
    
    return abs(τ1)

# Margolus-Levitin Bound
def margolus(Ham,ψ0,ψ1):
    e_min = min(list(Ham.eigenvals().keys()))
    n = sp.shape(Ham)[0]
    I_n = eye(n)
    H = Ham - e_min*I_n
    expec_Ham = expec(H,ψ0)
    angle_squared = (angle(ψ0,ψ1))**(2)
    
    τ2 = 2/(sp.pi*expec_Ham) * angle_squared
    
    return abs(τ2).evalf()



# We want the target states in a list!
def final_state_list(target_gate):
    L = []
    n = len((target_gate).col(0))
    
    #print('LENGTH IS')
    #print(n)
    
    for i in range(n):
        final_state_i = final_state(target_gate,i)
        L.append(final_state_i)
        
    return L


# for all state transitions \psi_0_basis to \psi_0_target,
# we calculate the speed limit and put them in a horny list!
def mandelstamm_list_1(Ham,basis,target_basis):
    L = []
    
    for i in range(len(basis)):
        ψ0 = basis[i]
        ψ1 = target_basis[i]
        speed_limit_i = state_speed_limit_1(Ham,ψ0,ψ1)
        
        L.append(speed_limit_i)
        

    
    max_speed_limit = max(L)
    print(max_speed_limit)
    
    
    return L


def margolus_list_2(Ham,basis,target_basis):
    L = []
    
    for i in range(len(basis)):
        ψ0 = basis[i]
        ψ1 = target_basis[i]
        speed_limit_i = state_speed_limit_2(Ham,ψ0,ψ1)
        
        L.append(speed_limit_i)
        
    
    max_speed_limit = max(L)
    print(max_speed_limit)
    
    return L 

In [24]:
# Identity
E = mx([[1,0],[0,1]])

# Pauli Gates
X = mx([[0,1],[1,0]])
Y = mx([[0,-I],[I,0]])
Z = mx([[1,0],[0,-1]])

### Single Qubit Basis

ket_0 = mx([1,0])
ket_1 = mx([0,1])

sup_state_1 = 1/sp.sqrt(2)*(ket_0  + ket_1)
sup_state_2 = 1/sp.sqrt(2)*(ket_0  - ket_1)


### Two Qubit Basis

k_00 = tp(ket_0,ket_0)
k_01 = tp(ket_0,ket_1)
k_10 = tp(ket_1,ket_0)
k_11 = tp(ket_1,ket_1)

final_state_0 = (1/sp.sqrt(2))*(k_00 + k_11)
final_state_T = (1/sp.sqrt(2))*(k_00 - k_11)


### Three Qubit Basis

k_000 = tp(ket_0,ket_0,ket_0)
k_010 = tp(ket_0,ket_1,ket_0)
k_001 = tp(ket_0,ket_0,ket_1)
k_011 = tp(ket_0,ket_1,ket_1)

k_100 = tp(ket_1,ket_0,ket_0)
k_110 = tp(ket_1,ket_1,ket_0)
k_101 = tp(ket_1,ket_0,ket_1)
k_111 = tp(ket_1,ket_1,ket_1)

sup_ent_state_0 = (1/sp.sqrt(2))*(k_000 + k_111)
sup_ent_state_T = (1/sp.sqrt(2))*(k_000 -  k_111)

# Single Qubit described by H_sq

In [5]:
ω = sp.Symbol('ω', real=True, positive = True)
#ω = 2*np.pi*5.0

H_sq = -ω/2*Z

In [6]:
mandelstamm(H_sq,sup_state_1,sup_state_2)

3.14159265358979/ω

In [7]:
margolus(H_sq,sup_state_1,sup_state_2)

3.14159265358979/ω

The Mandelstamn-Tamm Bound and the Margolus-Levitin Bound yield identical results, indicating that the qubit described by H_sq evolves at the fastest possible rate, or in other words, at the quantum speed limit.

# Two interacting qubits are described by the Hamiltonian H_{2q}

In [8]:
ω1 = 5.0 
ω2 = 5.3
κ1 = 0.112

#Ham
H_2q = (-ω1/2)*tp(Z,E) + (-ω2/2)*tp(E,Z) + κ1*tp(X,X)


In [9]:
mandelstamm(H_2q,final_state_0,final_state_T)

0.305008995494155

In [10]:
margolus(H_2q,final_state_0,final_state_T)

0.298447909686957

Furthermore, we show that the result will not change by integrating additional qubit-qubit interactions

In [11]:
κ2 = 0.222

H_xz = H_2q + κ2*tp(Z,Z)

In [12]:
mandelstamm(H_xz,final_state_0,final_state_T)

0.305008995494155

In [13]:
margolus(H_xz,final_state_0,final_state_T)

0.298447909686957

Here, we investigate the asymmetric three qubit chain described by the Hamiltonian H_3q. The speed limit will be determined

In [14]:
# Three-Qubit Hamiltonian

ω3 = 5.5 
κ3 = 0.314


# The drift describes the energy of the system, as a sum of all qubit energies
drift_3qubit = (-ω1/2)*tp(Z,E,E) + (-ω2/2)*tp(E,Z,E) + (-ω3/2 * tp(E,E,Z))


# Couplings between the qubits, in this case CHAIN COUPLINGS
coupl_3qubit =  κ1*tp(X,X,E) + κ2*tp(Z,Z,E) + κ3*tp(E,X,X)



# Total Hamiltonian as a sum of DRIFT and COUPLING terms
Ham_3qub = drift_3qubit + coupl_3qubit




In [15]:
mandelstamm(Ham_3qub,sup_ent_state_0,sup_ent_state_T)

0.198658171086516

In [16]:
margolus(Ham_3qub,sup_ent_state_0,sup_ent_state_T)

0.198565230796525

# State Transformation in Landau-Zener Model

In 2009, T. Caneva and colleagues demonstrated that quantum systems could be guided in accordance with the quantum speed limit by designing a control protocol that minimized the evolution time of a quantum system within the specified constraints (see 
<font color = blue> https://journals.aps.org/prl/pdf/10.1103/PhysRevLett.103.240501 </font>).


Their findings indicated that the calculated quantum speed limit was consistent with the numerical results obtained through optimal control theory (OCT). They illustrated this by examining state transformation in a system modeled by the Landau-Zener Hamiltonian.





In [17]:
# Landau Zener Hamiltonian

Γ = sp.Symbol('Γ(t)',real=True)

H_LZ = ω*X + Γ*Z
H_LZ

Matrix([
[Γ(t),     ω],
[   ω, -Γ(t)]])

Here, we illustrate how the analytical result was obtained.

In [18]:
# get eigenvectors
vec_1_inf,vec_2_inf = H_LZ.eigenvects()

v1 = vec_1_inf[2][0]
v2 = vec_2_inf[2][0]

v1, v2

# get eigenstates, normalize eigenvectors
v_min= 1/(v1.norm())*v1
v_plus= 1/(v2.norm())*v2

v_min,v_plus

# get instantaneous ground state for t=0 and t=T
vm_0 = v_min.evalf(subs={Γ :-500*ω})
vm_0

vm_T = v_min.evalf(subs={Γ :500*ω})
vm_T


Matrix([
[-0.000999998499988804],
[    0.999999500001375]])

In [19]:
# get state infidelity

inner_prod = ((vm_T.T* vm_0)[0]).simplify()

I = 1-inner_prod

In [20]:
Mand = mandelstamm(H_LZ,vm_0,vm_T)

Mand

1.56879946705109/Abs(sqrt(3.99999999984986e-6*Γ(t)**2 + 0.004*Γ(t)*ω + ω**2))

In [21]:
Marg = margolus(H_LZ,vm_0,vm_T)

Marg

1.56679887860055/Abs(0.999998000006*Γ(t) - 0.001999996000012*ω + 1.0*sqrt(Γ(t)**2 + ω**2))